# Defining Tools: Giving Your Agent Capabilities

## Introduction

In this notebook, you'll learn how to define tools that give your agent real capabilities beyond just conversation. Tools allow the LLM to take actions, retrieve data, and interact with external systems.

### What You'll Learn

- What tools are and why they're essential for agents
- How to define tools with proper schemas
- How the LLM knows which tool to use
- How tool descriptions affect LLM behavior
- Best practices for tool design

### Prerequisites

- Completed `01_system_instructions.ipynb`
- Redis 8 running locally
- OpenAI API key set
- Course data ingested (from Section 1)

## Concepts: Tools for AI Agents

### What Are Tools?

Tools are **functions that the LLM can call** to perform actions or retrieve information. They extend the agent's capabilities beyond text generation.

**Without tools:**
- Agent can only generate text based on its training data
- No access to real-time data
- Can't take actions
- Limited to what's in the prompt

**With tools:**
- Agent can search databases
- Agent can retrieve current information
- Agent can perform calculations
- Agent can take actions (send emails, create records, etc.)

### How Tool Calling Works

1. **LLM receives** user query + system instructions + available tools
2. **LLM decides** which tool(s) to call (if any)
3. **LLM generates** tool call with parameters
4. **System executes** the tool function
5. **Tool returns** results
6. **LLM receives** results and generates response

### Tool Schema Components

Every tool needs:
1. **Name** - Unique identifier
2. **Description** - What the tool does (critical for selection!)
3. **Parameters** - Input schema with types and descriptions
4. **Function** - The actual implementation

### How LLMs Select Tools

The LLM uses:
- Tool **names** (should be descriptive)
- Tool **descriptions** (should explain when to use it)
- Parameter **descriptions** (should explain what each parameter does)
- **Context** from the conversation

**Key insight:** The LLM only sees the tool schema, not the implementation!

## Setup

In [3]:
# Environment setup
import os
from typing import List, Optional
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")

print("🔧 Environment Setup")
print("=" * 30)
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Not set'}")
print(f"Redis URL: {REDIS_URL}")

🔧 Environment Setup
OpenAI API Key: ✅ Set
Redis URL: redis://localhost:6379


In [4]:
import os
from typing import List, Optional
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# Import our course manager
from redis_context_course import CourseManager

# Initialize
llm = ChatOpenAI(model="gpt-4o", temperature=0)
course_manager = CourseManager()

print("✅ Setup complete!")

16:39:08 redisvl.index.index INFO   Index already exists, not overwriting.
✅ Setup complete!


## Hands-on: Defining Tools

Let's define tools for our class agent step by step.

### Tool 1: Search Courses (Basic)

Let's start with a basic tool to search courses.

In [5]:
# Define parameter schema
class SearchCoursesInput(BaseModel):
    query: str = Field(description="Search query for courses")
    limit: int = Field(default=5, description="Maximum number of results")

# Define the tool
@tool(args_schema=SearchCoursesInput)
async def search_courses_basic(query: str, limit: int = 5) -> str:
    """Search for courses in the catalog."""
    results = await course_manager.search_courses(query, limit=limit)
    
    if not results:
        return "No courses found matching your query."
    
    output = []
    for course in results:
        output.append(
            f"{course.course_code}: {course.title}\n"
            f"  Credits: {course.credits} | {course.format.value}\n"
            f"  {course.description[:100]}..."
        )
    
    return "\n\n".join(output)

print("Tool defined:", search_courses_basic.name)
print("Description:", search_courses_basic.description)

Tool defined: search_courses_basic
Description: Search for courses in the catalog.


**Problem:** The description is too vague! The LLM won't know when to use this tool.

### Tool 1: Search Courses (Improved)

Let's improve the description to help the LLM understand when to use this tool.

In [6]:
@tool(args_schema=SearchCoursesInput)
async def search_courses(query: str, limit: int = 5) -> str:
    """
    Search for courses in the Redis University catalog using semantic search.
    
    Use this tool when students ask about:
    - Finding courses on a specific topic (e.g., "machine learning courses")
    - Courses in a department (e.g., "computer science courses")
    - Courses with specific characteristics (e.g., "online courses", "3-credit courses")
    
    The search uses semantic matching, so natural language queries work well.
    """
    results = await course_manager.search_courses(query, limit=limit)
    
    if not results:
        return "No courses found matching your query."
    
    output = []
    for course in results:
        output.append(
            f"{course.course_code}: {course.title}\n"
            f"  Credits: {course.credits} | {course.format.value} | {course.difficulty_level.value}\n"
            f"  {course.description[:150]}..."
        )
    
    return "\n\n".join(output)

print("✅ Improved tool defined!")
print("\nDescription:")
print(search_courses.description)

✅ Improved tool defined!

Description:
Search for courses in the Redis University catalog using semantic search.

Use this tool when students ask about:
- Finding courses on a specific topic (e.g., "machine learning courses")
- Courses in a department (e.g., "computer science courses")
- Courses with specific characteristics (e.g., "online courses", "3-credit courses")

The search uses semantic matching, so natural language queries work well.


### Tool 2: Get Course Details

A tool to get detailed information about a specific course.

In [7]:
class GetCourseDetailsInput(BaseModel):
    course_code: str = Field(description="Course code (e.g., 'CS101', 'MATH201')")

@tool(args_schema=GetCourseDetailsInput)
async def get_course_details(course_code: str) -> str:
    """
    Get detailed information about a specific course by its course code.
    
    Use this tool when:
    - Student asks about a specific course (e.g., "Tell me about CS101")
    - You need prerequisites for a course
    - You need full course details (schedule, instructor, etc.)
    
    Returns complete course information including description, prerequisites,
    schedule, credits, and learning objectives.
    """
    course = await course_manager.get_course(course_code)
    
    if not course:
        return f"Course {course_code} not found."
    
    prereqs = "None" if not course.prerequisites else ", ".join(
        [f"{p.course_code} (min grade: {p.min_grade})" for p in course.prerequisites]
    )
    
    return f"""
{course.course_code}: {course.title}

Description: {course.description}

Details:
- Credits: {course.credits}
- Department: {course.department}
- Major: {course.major}
- Difficulty: {course.difficulty_level.value}
- Format: {course.format.value}
- Prerequisites: {prereqs}

Learning Objectives:
""" + "\n".join([f"- {obj}" for obj in course.learning_objectives])

print("✅ Tool defined:", get_course_details.name)

✅ Tool defined: get_course_details


### Tool 3: Check Prerequisites

A tool to check if a student meets the prerequisites for a course.

In [8]:
class CheckPrerequisitesInput(BaseModel):
    course_code: str = Field(description="Course code to check prerequisites for")
    completed_courses: List[str] = Field(
        description="List of course codes the student has completed"
    )

@tool(args_schema=CheckPrerequisitesInput)
async def check_prerequisites(course_code: str, completed_courses: List[str]) -> str:
    """
    Check if a student meets the prerequisites for a specific course.
    
    Use this tool when:
    - Student asks "Can I take [course]?"
    - Student asks about prerequisites
    - You need to verify eligibility before recommending a course
    
    Returns whether the student is eligible and which prerequisites are missing (if any).
    """
    course = await course_manager.get_course(course_code)
    
    if not course:
        return f"Course {course_code} not found."
    
    if not course.prerequisites:
        return f"✅ {course_code} has no prerequisites. You can take this course!"
    
    missing = []
    for prereq in course.prerequisites:
        if prereq.course_code not in completed_courses:
            missing.append(f"{prereq.course_code} (min grade: {prereq.min_grade})")
    
    if not missing:
        return f"✅ You meet all prerequisites for {course_code}!"
    
    return f"""❌ You're missing prerequisites for {course_code}:

Missing:
""" + "\n".join([f"- {p}" for p in missing])

print("✅ Tool defined:", check_prerequisites.name)

✅ Tool defined: check_prerequisites


## Testing: Using Tools with an Agent

Let's test our tools with the LLM to see how it selects and uses them.

In [10]:
# Bind tools to LLM
tools = [search_courses, get_course_details, check_prerequisites]
llm_with_tools = llm.bind_tools(tools)

# System prompt
system_prompt = """You are the Redis University Class Agent.
Help students find courses and plan their schedule.
Use the available tools to search courses and check prerequisites.
"""

print("✅ Agent configured with tools!")

✅ Agent configured with tools!


### Test 1: Search Query

In [11]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="I'm interested in machine learning courses")
]

response = llm_with_tools.invoke(messages)

print("User: I'm interested in machine learning courses")
print("\nAgent decision:")
if response.tool_calls:
    for tool_call in response.tool_calls:
        print(f"  Tool: {tool_call['name']}")
        print(f"  Args: {tool_call['args']}")
else:
    print("  No tool called")
    print(f"  Response: {response.content}")

16:40:48 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User: I'm interested in machine learning courses

Agent decision:
  Tool: search_courses
  Args: {'query': 'machine learning', 'limit': 5}


### Test 2: Specific Course Query

In [ ]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="Tell me about CS401")
]

response = llm_with_tools.invoke(messages)

print("User: Tell me about CS401")
print("\nAgent decision:")
if response.tool_calls:
    for tool_call in response.tool_calls:
        print(f"  Tool: {tool_call['name']}")
        print(f"  Args: {tool_call['args']}")
else:
    print("  No tool called")
    print(f"  Response: {response.content}")

### Test 3: Prerequisites Query

In [ ]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="Can I take CS401? I've completed CS101 and CS201.")
]

response = llm_with_tools.invoke(messages)

print("User: Can I take CS401? I've completed CS101 and CS201.")
print("\nAgent decision:")
if response.tool_calls:
    for tool_call in response.tool_calls:
        print(f"  Tool: {tool_call['name']}")
        print(f"  Args: {tool_call['args']}")
else:
    print("  No tool called")
    print(f"  Response: {response.content}")

## Key Takeaways

### Tool Design Best Practices

1. **Clear Names**
   - Use descriptive, action-oriented names
   - `search_courses` ✅ vs. `find` ❌

2. **Detailed Descriptions**
   - Explain what the tool does
   - Explain when to use it
   - Include examples

3. **Well-Defined Parameters**
   - Use type hints
   - Add descriptions for each parameter
   - Set sensible defaults

4. **Useful Return Values**
   - Return formatted, readable text
   - Include relevant details
   - Handle errors gracefully

5. **Single Responsibility**
   - Each tool should do one thing well
   - Don't combine unrelated functionality

### How Tool Descriptions Affect Selection

The LLM relies heavily on tool descriptions to decide which tool to use:

- ✅ **Good description**: "Search for courses using semantic search. Use when students ask about topics, departments, or course characteristics."
- ❌ **Bad description**: "Search courses"

**Remember:** The LLM can't see your code, only the schema!

## Exercises

1. **Add a new tool** called `get_courses_by_department` that returns all courses in a specific department. Write a good description.

2. **Test tool selection**: Create queries that should trigger each of your three tools. Does the LLM select correctly?

3. **Improve a description**: Take the `search_courses_basic` tool and improve its description. Test if it changes LLM behavior.

4. **Create a tool** for getting a student's current schedule. What parameters does it need? What should it return?

## Summary

In this notebook, you learned:

- ✅ Tools extend agent capabilities beyond text generation
- ✅ Tool schemas include name, description, parameters, and implementation
- ✅ LLMs select tools based on descriptions and context
- ✅ Good descriptions are critical for correct tool selection
- ✅ Each tool should have a single, clear purpose

**Next:** In Section 3, we'll add memory to our agent so it can remember user preferences and past conversations.